<a href="https://colab.research.google.com/github/MatteoRigoni/AI-Agent-workshop/blob/master/AI_Agent_example_with_local_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#unsloth: to load advanced LLM models
#transformers: to tokenize
#accelerate e bitsandbytes: to optimize perfomance
!pip install unsloth transformers accelerate bitsandbytes

In [2]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
#Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length=8192, #max sequence length
    load_in_4bit=True #load model with format 4 bit, to reduce memory
)

==((====))==  Unsloth 2025.8.5: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [4]:
#TextStreamer from Hugging FAce, to show real time generated output
from transformers import TextStreamer

In [5]:
#Set chat template
from unsloth.chat_templates import get_chat_template

In [6]:
#Configuring tokenizer and fields mapping
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
    mapping={"role":"role", "content":"content", "user":"human", "assistant": "gpt"}
)

In [7]:
text_streamer = TextStreamer(tokenizer) #to have stream real-time of output text

In [9]:
#Enable inference mode to speed up the model 2x
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((409

In [17]:
#Function to send prompt to model via tokenizer
def prompt_model(messages):
  input_ids  = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to("cuda")

  print(f"Structure of inputs: {input_ids}")
  print(f"Size of inputs: {input_ids.shape}")

  response = model.generate(
      input_ids =input_ids,
      streamer=text_streamer,
      max_new_tokens=1024,
      use_cache=True
  )

  #decode response
  response_text = tokenizer.decode(response[0], skip_special_tokens=True)
  print(f"Response text: {response_text}")

  return response_text

In [11]:
#Function to do the booking
def interactive_booking():
  print("Welcome to out booking system powered by Meta-Llama-3.1-8B-Instruct")
  chat_history=[]

  while True:
    request = input("Insert your booking request (or digit 'exit')")
    if request.lower() == "exit":
      print("Thanks for using our system. Bye.")
      break

    print(f"\n**Cliente:** \"{request}\"")
    chat_history.append({"role":"human", "content":request})

    model_response = prompt_model(chat_history)
    print(f"\n**Restaurant:** \"{model_response}\"")
    chat_history.append({"role":"assistant", "content":model_response})

    print("-" * 50)

In [ ]:
interactive_booking()

Welcome to out booking system powered by Meta-Llama-3.1-8B-Instruct
Insert your booking request (or digit 'exit')tavolo per 5


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



**Cliente:** "tavolo per 5"
Structure of inputs: tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,   5887,    220,   2366,     19,    271, 128009, 128006,
          26380, 128007,    271,     83,    402,  10216,    824,    220,     20,
         128009, 128006,  78191, 128007,    271]], device='cuda:0')
Size of inputs: torch.Size([1, 41])
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

tavolo per 5<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Vorresti organizzare un tavolo per 5 persone? Ecco alcune idee per un'atmosfera accogliente e funzionale:

1. **Scelta del tavolo**: Scegli un tavolo di dimensioni adeguate per 5 persone, con una superficie ampia e comoda per mangiare e conversare.
2. **Posizionamento**: Pos